In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
#input_file = 'mariel-1.npy'
#input_file = 'carrie-10-min-scaled.npy'
input_file = 'carrie-10-mins.npy'

X0 = np.load(os.path.join('data','npy', input_file))

if input_file == 'mariel-1.npy':
    X0 = X0.transpose((1,0,2))
    X0 = X0[1300:7000]
    X0[:,:,2] *= -1
    X = X0.copy()
    X -= X.mean(axis=(0,1))
    X *= 0.5
elif input_file == 'carrie-10-min-scaled.npy':
    X0 = X0.transpose((1,0,2))
    X = X0.copy()
    X[:,:,1] = X0[:,:,2]
    X[:,:,2] = X0[:,:,1]
    X = X[100:]
    X[:,:,:2] -= 0.5
if input_file == 'carrie-10-mins.npy':
    X0 = X0.transpose((1,0,2))
    X0 = X0[100:]
    X0[:,:,2] *= -1
    X = X0.copy()
    X -= X.mean(axis=(0,1))
    X *= 0.5
    

do_cluster = False
if do_cluster:
    from sklearn.cluster import KMeans
    
    nc = 8
    km = KMeans(n_clusters=nc).fit(X.transpose((1,2,0)).reshape(X.shape[1],-1))
    Xcl = np.zeros((X.shape[0], nc, 3))
    for i in range(nc):
        Xcl[:,i] = X[:,np.argwhere(km.labels_==i)[:,0]].mean(axis=1)
    X = Xcl
    
print(X.shape)

In [ ]:
plt.plot(X.mean(axis=1));
plt.figure()
plt.hist([X[:,:,0].flatten(),X[:,:,1].flatten(),X[:,:,2].flatten()], histtype='step');

In [ ]:
print(X.max(axis=(0,1)))
print(X.min(axis=(0,1)))

In [ ]:
print(X.mean(axis=(0,1)))
print(X.std(axis=(0,1)))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.mplot3d.art3d import juggle_axes
from matplotlib import animation
from IPython.display import HTML

# ask matplotlib to plot up to 2^128 frames in animations
matplotlib.rcParams['animation.embed_limit'] = 2**128

def update_points(time, points, df):
  '''
  Callback function called by plotting function below. Mutates the vertex
  positions of each value in `points` so the animation moves
  @param int time: the index of the time slice to visualize within `df`
  @param mpl_toolkits.mplot3d.art3d.Path3DCollection points: the actual
    geometry collection whose internal values this function mutates to move
    the displayed points
  @param numpy.ndarray df: a numpy array with the following three axes:
    df.shape[0] = n_vertices
    df.shape[1] = n_time_slices
    df.shape[2] = n_dimensions
  '''
  points._offsets3d = juggle_axes(df[time,:,0], df[time,:,1], df[time,:,2], 'z')

def animate(seq, frames=None, axis_min=-.75, axis_max=0.75, speed=45):
    if frames is None:
        frames = len(seq)
    fig = plt.figure()
    ax = p3.Axes3D(fig)
    ax.set_xlim(axis_min, axis_max)
    ax.set_ylim(axis_min, axis_max)
    ax.set_zlim(axis_min, axis_max)
    points = ax.scatter(seq[0,:,0], seq[0,:,1], seq[0,:,2], depthshade=False, marker='.')
    return animation.FuncAnimation(
        fig,
        update_points,
        frames,
        interval=speed,
        fargs=(points, seq),
        blit=False,
    ).to_jshtml()

def plot_pose(x, ax=None, fig=None, subplot=None,
              xlim=(-0.75,0.75), ylim=(-0.75,0.75), zlim=(-0.75, 0.75), xylim=None, lim=None, center=0):
    if fig is None:
        if ax:
            fig = ax.get_figure()
        else:
            fig = plt.figure()
    if not ax:
        if subplot:
            ax = fig.add_subplot(*subplot, projection='3d')
        else:
            ax = p3.Axes3D(fig)
    if xylim:
        xlim = ylim = xylim
    if lim:
        xlim = ylim = zlim = lim
    if center:
        x0, x1 = x.min(), x.max()
        xlim = ylim = zlim = (center*x0, center*x1)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(*zlim)
    ax.scatter(x[:,0], x[:,1], x[:,2], marker='.')
    return ax

def plot_stick(x, edges, fig=None, ax=None, subplot=None,
               xlim=(-0.75,0.75), ylim=(-0.75,0.75), zlim=(-0.75, 0.75), xylim=None, lim=None,
               center=0):
    if fig is None:
        if ax:
            fig = ax.get_figure()
        else:
            fig = plt.figure()
    if ax is None:
        if subplot:
            ax = fig.add_subplot(*subplot, projection='3d')
        else:
            ax = p3.Axes3D(fig)
    if xylim:
        xlim = ylim = xylim
    if lim:
        xlim = ylim = zlim = lim
    if center:
        x0, x1 = x.min(), x.max()
        xlim = ylim = zlim = (center*x0, center*x1)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_zlim(*zlim)
    for e in edges:
        ax.plot(np.linspace(x[e[0],0],x[e[1],0],10),
                np.linspace(x[e[0],1],x[e[1],1],10),
                np.linspace(x[e[0],2],x[e[1],2],10)
               )
    return ax

In [ ]:
HTML(animate(X[100:300]))

In [ ]:
vdist_mean = np.zeros((X.shape[1],X.shape[1]))
vdist_var = np.zeros_like(vdist_mean)
for ivtx in range(X.shape[1]):
    for jvtx in range(ivtx+1,X.shape[1]):
        vdist = np.sum((X[:,ivtx]-X[:,jvtx])**2, axis=-1)
        vdist_mean[ivtx,jvtx] = vdist_mean[jvtx,ivtx] = vdist.mean()
        vdist_var[ivtx,jvtx] = vdist_var[jvtx,ivtx] = vdist.var(ddof=1)
plt.figure()
plt.subplot(1,2,1)
plt.title("mean")
plt.imshow(vdist_mean)
plt.subplot(1,2,2)
plt.title("var")
plt.imshow(vdist_var)

In [ ]:
upper_triangle = np.triu_indices_from(vdist_var, k=1)
variances = vdist_var[upper_triangle]
vtx_pairs = sorted(zip(*upper_triangle), key=lambda p: vdist_var[p[0],p[1]])

In [ ]:
n_rigid = 140
#n_rigid = 7
fig = plt.figure(figsize=(10,2))
plt.plot(sorted(vdist_var[upper_triangle]))
plt.yscale('log')
plt.ylabel('variance', fontsize=14)
plt.xlabel('edge #', fontsize=14)
plt.axvline(n_rigid, color='red', lw=1);
#plt.subplot(2,1,2,)
#plt.figure(figsize=(14,14))
frame_num = 500
fig = plt.figure(figsize=(8,8))
ax = plot_stick(X[frame_num], vtx_pairs[:n_rigid], fig=fig, center=1)
plot_pose(X[frame_num], ax, center=1);
plt.title("Reference pose (frame %d, %d edges)"%(frame_num, n_rigid), fontsize=16);

In [ ]:
import os
import tensorflow as tf
# config for Gatsby cores
if 'gatsby' in os.environ['HOSTNAME']:

    # specify target gpu device
    os.environ['CUDA_VISIBLE_DEVICES'] = '1' # either '0' or '1' to utilize Titan X GPUs

    # allow dynamic GPU allocation
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    
import keras.backend as K
from keras import layers
from keras.models import Model
from keras import objectives

In [ ]:
def mk_model(lookback=60, npred=1, n_cells=(32,32,32), center_seq=True, n_rigid=0, do_rotations=False):
    K.clear_session()
    probe = None
    
    input_layer = layers.Input((lookback, X.shape[1], X.shape[2]))
    H = input_layer
    
    if do_rotations:
        pass
    
    if center_seq:
        # mask [1,1,0] to ignore z axis shifts
        offsets = layers.Lambda(lambda x: K.constant([[[[1,1,0]]]])*K.mean(x,axis=(1,2),keepdims=True))(H)
        H = layers.Subtract()([H,offsets])
        probe = Model(input_layer, H)
    
    H = layers.Reshape((lookback, X.shape[1]*X.shape[2]))(H)
    
    for nc in n_cells:
        H = layers.CuDNNLSTM(nc, return_sequences=True)(H)
    #H = layers.CuDNNLSTM(n_cells[-1])(H)
    
    H = layers.Cropping1D((lookback-npred,0))(H)
    H = layers.Dense(X.shape[1]*X.shape[2], activation='tanh')(H)
    
    #H = layers.LeakyReLU()(layers.Dense(npred*n_cells[-1])(H))
    #H = layers.Reshape((npred, n_cells[-1]))(H)
    #H = layers.CuDNNLSTM(X.shape[1]*X.shape[2], return_sequences=True)(H)
    
    #H = layers.Dense(npred*X.shape[1]*X.shape[2], activation='tanh')(H)
    H = layers.Reshape((npred,X.shape[1],X.shape[2]))(H)
    
    if center_seq:
        H = layers.Add()([H,offsets])
    
    output_layer = H
    
    model = Model(input_layer, output_layer)
    
    model.hp_mse_weight = K.variable(1.0)
    model.hp_rigidity_weight = K.variable(1.0)
    
    def loss(y_true, y_pred):
        rigid_loss = 0
        for i,j in vtx_pairs[:n_rigid]:
            a = output_layer[:,:,i]
            b = output_layer[:,:,j]
            D = K.sum(K.square(a-b), axis=-1)
            d = vdist_mean[i,j]
            v = vdist_var[i,j]
            rigid_loss += K.square(D-d)/v
        if n_rigid:
            rigid_loss /= n_rigid
        
        return model.hp_mse_weight*K.mean((y_true-y_pred)**2, axis=(-2,-1)) \
             + model.hp_rigidity_weight*rigid_loss
    
    #model.compile(loss=loss, optimizer='adam')
    model.compile(loss=loss, optimizer='adam')
    
    return model, probe

In [ ]:
lookback = 32
npred = 1
center_seq = True
n_cells = (64,64,64,)

model,probe = mk_model(lookback, npred, n_cells=n_cells, center_seq=center_seq, n_rigid=n_rigid)
model.summary()

losses = []

In [ ]:
K.set_value(model.optimizer.lr, 4e-4)

K.set_value(model.hp_mse_weight, 1)
K.set_value(model.hp_rigidity_weight, 4e-6)

In [ ]:
# bs=128, lr=1e-4, no rotation should get interesting results in about 600 epochs
epochs = 1
batch_size = 128
do_rotation = True

nbatches = (len(X)-batch_size*lookback-npred)//batch_size

idxs = np.arange(len(X))
#nbatches = (len(X)-1)//(batch_size*lookback)
print("Nbatches =", nbatches)
for iepoch in range(epochs):
    avg_loss = 0
    offsets = np.random.choice(len(X)-batch_size*lookback-npred, replace=False, size=(nbatches,batch_size))
    for ibatch in range(nbatches):
        batch_idxs = offsets[ibatch].repeat(lookback).reshape(batch_size,lookback) + np.arange(lookback)
        truth_idxs = offsets[ibatch].repeat(npred).reshape(batch_size,npred) + np.arange(npred) + lookback
        
        if do_rotation:
            theta = 2*np.pi*np.random.rand()
            c,s = np.cos(theta),np.sin(theta)
            rot = np.array([[c,-s,0],[s,c,0],[0,0,1]])

            x_train = rot.dot(X[batch_idxs].transpose((0,1,3,2))).transpose((1,2,3,0))
            y_train = rot.dot(X[truth_idxs].transpose((0,1,3,2))).transpose((1,2,3,0))
        else:
            x_train = X[batch_idxs]
            y_train = X[truth_idxs]
            
        avg_loss += model.train_on_batch(x_train, y_train)
    losses.append(avg_loss/nbatches)
    print("Epoch %d/%d: L=%.3e"%(iepoch,epochs,losses[-1]))

In [ ]:
plt.plot(losses)
plt.yscale('log')

In [ ]:
#model.save('models/rnn-model.h5')
#model.save_weights('models/rnn-e%03d.wt.h5'%len(losses))
#model.load_weights('models/rnn-e2405.wt.h5')

In [ ]:
iprompt = np.random.randint(len(X)-batch_size*lookback-1)
ngen = 8*lookback

gen_seq = np.zeros((lookback+ngen, X.shape[1], X.shape[2]))
gen_seq[:lookback] = X[iprompt:iprompt+lookback]

for i in range(ngen):
    next_frame = model.predict(np.expand_dims(gen_seq[i:i+lookback],0))[0,0]
    gen_seq[i+lookback] = next_frame

ax = plot_stick(gen_seq[lookback], vtx_pairs[:n_rigid])
plot_pose(gen_seq[lookback], ax)
plt.title("Epoch %d" % (len(losses)-1))
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.title('real')
plt.hist([gen_seq[:lookback,:,0].flatten(),
          gen_seq[:lookback,:,1].flatten(),
          gen_seq[:lookback,:,2].flatten()], histtype='step', bins=20, range=(-1.5,1.5));
plt.subplot(1,2,2)
plt.title('gen')
plt.hist([gen_seq[lookback:,:,0].flatten(),
          gen_seq[lookback:,:,1].flatten(),
          gen_seq[lookback:,:,2].flatten()], histtype='step', bins=20, range=(-1.5,1.5));
    
dm = np.zeros_like(vdist_mean)
dv = np.zeros_like(vdist_var)
for i in range(X.shape[1]):
    for j in range(i+1,X.shape[1]):
        dm[i,j] = dm[j,i] = np.sqrt((gen_seq[lookback:,i]-gen_seq[lookback:,j])**2).sum(axis=-1).mean()
        dv[i,j] = dv[j,i] = np.sqrt((gen_seq[lookback:,i]-gen_seq[lookback:,j])**2).sum(axis=-1).var(ddof=1)
        
plt.figure()
plt.subplot(1,2,1)
plt.imshow(vdist_mean, vmin=0, vmax=vdist_mean.max())
plt.title('real')
plt.subplot(1,2,2)
plt.title('gen')
plt.imshow(dm, vmin=0, vmax=vdist_mean.max());
plt.figure()
plt.subplot(1,2,1)
plt.imshow(np.sqrt(vdist_var), vmin=0, vmax=np.sqrt(vdist_var).max())
plt.title('real')
plt.subplot(1,2,2)
plt.title('gen')
plt.imshow(np.sqrt(dv), vmin=0, vmax=np.sqrt(vdist_var).max());

rigidity_score = 0
for i,j in vtx_pairs[:n_rigid]:
    rigidity_score += ((np.sum((gen_seq[lookback:,i]-gen_seq[lookback:,j])**2,axis=-1)-vdist_mean[i,j])**2)/vdist_var[i,j]
if n_rigid:
    rigidity_score /= n_rigid
print("Rigidity score: %.2e"%rigidity_score.mean())

In [ ]:
HTML(animate(gen_seq))